# Background {#sec-background}


## Ritchie Vink, Rust, Apache Arrow and Covid

[Here](https://www.ritchievink.com/blog/2021/02/28/i-wrote-one-of-the-fastest-dataframe-libraries/) is the story, by the creator of Polars.


## Who Can Benefit from Polars?

- Researcher (DS, Analyst, Statistician, etc):
  - Working on their local machine. 
  - Working on a cloud machine (SageMaker, EC2).
- Production system:
  - Running on a dedicated server. 
  - Running on "serverless" (e.g. AWS Lambda, Google Cloud Functions).

## The DataFrame Landscape

Initially there were R's `data.frame`. 
R has evolved, and it now offers `tibble`s and `data.table`s.
Python had only `Pandas` for years. 
Then the Python ecosystem exploded, and now we have:

-  [Pandas](https://Pandas.pydata.org/): The original Python dataframe module. Build by Wes McKinney, on top of numpy.
-  [Polars](https://www.pola.rs/): A new dataframe module, build by Ritchie Vink, on top of Rust and Apache Arrow.
-  [datatable](https://datatable.readthedocs.io/en/latest/): An attempt to recreate R's [data.table](https://github.com/Rdatatable/data.table) API and (crazy) speed in Python. 
-  [Dask](https://www.dask.org/): A distributed computing engine for Python, with support for distributing data over multiple processes running Pandas (or numpy, Polars, etc).
-  [Vaex](https://vaex.io/): A high performance Python library for lazy Out-of-Core DataFrames (similar to dask, but with a different API).
-  [Modin](https://github.com/modin-project/modin): A drop-in distributed  replacement for Pandas, built on top of [Ray](https://www.ray.io/). 
-  [DuckDB](https://duckdb.org/): An embeddable SQL OLAP database management system. These are dataframe that are stored on disk, compute on a single process, and queried with SQL or pythonic API.
- [Daft](https://www.getdaft.io/): A distributed dataframe library built for "Complex Data" (data that doesn't usually fit in a SQL table such as images, videos, documents etc). 
-  [Fugue](https://fugue-tutorials.readthedocs.io/): A dataframe library that allows you to write SQL-like code, and execute it on different backends (e.g. Spark, Dask, Pandas, Polars, etc).
-  [pySpark](https://spark.apache.org/docs/latest/api/python/index.html): The Python API for Spark. Spark is a distributed computing engine, with support for distributing data over multiple processes running Pandas (or numpy, Polars, etc).
-  [CUDF](https://github.com/rapidsai/cudf): A GPU accelerated dataframe library, build on top of Apache Arrow.



See [here](https://pola-rs.github.io/Polars-book/user-guide/misc/alternatives/) and [here](https://www.getdaft.io/projects/docs/en/latest/dataframe_comparison.html) for more details. 





# Motivation {#sec-motivation}

Each of the following, alone(!), is amazing.

1. Small memory footprint.
1. Native dtypes: missing, strings.
2. Lazy evaluation allows query Planning.
3. Streaming engine: No need to load entire dataset into memory.
4. Out of the box parallelism: Fast and informative messages for debugging.
5. Strict typing: This means the dtype of output is defined by the operation and not bu the input. This is both safer, and allows static analysis.


## Setting Up the Environment

At this point you may want to create and activate a [venv](https://realpython.com/python-virtual-environments-a-primer/) for this project. 


In [1]:
# %pip install --upgrade pip
# %pip install --upgrade Polars
# %pip install --upgrade pyarrow
# %pip install --upgrade Pandas
# %pip install --upgrade plotly
# %pip freeze > requirements.txt
# %pip install -r requirements.txt

In [2]:
#| label: Polars-version
%pip show Polars # check you Polars version

In [3]:
#| label: Pandas-version
%pip show Pandas # check you Polars version

In [4]:
#| label: preliminaries

import polars as pl
pl.Config(fmt_str_lengths=50)

import pandas as pd
import numpy as np
import pyarrow as pa
import plotly.express as px
import string
import random
import os
import sys
%matplotlib inline 
import matplotlib.pyplot as plt
from datetime import datetime

# Following two lines only required to view plotly when rendering from VScode. 
import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected+notebook"
pio.renderers.default = "plotly_mimetype+notebook"

## Memory Footprint

### Memory Footprint of Storage

Comparing Polars to Pandas- the memory footprint of a series of strings. 

Polars. 

In [5]:
letters = pl.Series(list(string.ascii_letters))

n = int(10e6)
letter1 = letters.sample(n,with_replacement=True)
letter1.estimated_size(unit='gb') 

Pandas before Pandas 2.0. 

In [6]:
# Pandas with Ver 1.x backend
letter1_Pandas = letter1.to_pandas(use_pyarrow_extension_array=False) 
letter1_Pandas.memory_usage(deep=True, index=False) / 1e9

Pandas after Pandas 2.0, with the Pyarrow backend (Apr 2023).

In [7]:
letter1_Pandas = letter1.to_pandas(use_pyarrow_extension_array=True) 
letter1_Pandas.memory_usage(deep=True, index=False) / 1e9

### Operating From Disk to Disk

What if my data does not fit into RAM? 
Turns out you manifest a lazy frame into disk, instead of RAM, thus avoiding the need to load the entire dataset into memory. 
Alas, the function that does so, [sink_parquet()](https://pola-rs.github.io/Polars/py-Polars/html/reference/lazyframe/api/Polars.LazyFrame.sink_parquet.html), has currently limited functionality. 
It is certainly worth keeping an eye on this function, as it matures.

::: {.callout-note}
Although the potential of `sink_parquet()` is amazing, I find that it currently does not support complicated queries. 
:::


## Lazy Frames and Query Planning

Consider a sort operation that follows a filter operation. 
Ideally, filter precedes the sort, but we did not ensure this... 
We now demonstrate that Polars' query planner will do it for you. 
En passant, we see Polars is more efficient also without the query planner.


Polars' Eager evaluation in the **wrong** order. 
Sort then filter.


In [8]:
%timeit -n 2 -r 2 letter1.sort().filter(letter1.is_in(['a','b','c']))

Polars' Eager evaluation in the **right** order. 
Filter then sort.


In [9]:
%timeit -n 2 -r 2 letter1.filter(letter1.is_in(['a','b','c'])).sort()

In [10]:
latter1_lazy = letter1.alias('letters').to_frame().lazy()

Polars' Lazy evaluation in the **wrong** order; **without** query planning

In [11]:
%timeit -n 2 -r 2 latter1_lazy.sort(by='letters').filter(pl.col('letters').is_in(['a','b','c'])).collect(no_optimization=True)

Polars' Lazy evaluation in the **wrong** order; **with** query planning

In [12]:
%timeit -n 2 -r 2 latter1_lazy.sort(by='letters').filter(pl.col('letters').is_in(['a','b','c'])).collect()

Things to note:

1. A lazy evaluation was triggered when `df.lazy()` converted the Polars DataFrame to a Polars LazyFrame.
2. The query planner worked: The Lazy evaluation in the wrong order timed as much as an eager evaluation in the right order; even when accounting for the overhead of converting the frame from eager to lazy.

Now compare to Pandas...

Pandas' eager evaluation in the **wrong** order.


In [13]:
%timeit -n1 -r1 letter1_Pandas.sort_values().loc[lambda x: x.isin(['a','b','c'])]

Pandas eager evaluation in the **right** order: Filter then sort.


In [14]:
%timeit -n1 -r1 letter1_Pandas.loc[lambda x: x.isin(['a','b','c'])].sort_values()

Pandas without lambda functions syntax; looks slightly better.


In [15]:
%timeit -n 2 -r 2 letter1_Pandas.loc[letter1_Pandas.isin(['a','b','c'])].sort_values()

Things to note:

1. Query planning works!
2. Pandas has dramatically improved since <2.0.0. 
3. Lambda functions are always slow (both Pandas and Polars).


## Optimized for Within-Column Operations

Polars seamlessly parallelizes over columns (also within, when possible). 
As the number of columns in the data grows, we would expect fixed runtime until all cores are used, and then linear scaling. 
The following code demonstrates this idea, using a simple sum-within-column.


In [16]:
# Mac users with Apple silicon (M1 or M2) may also want to benchmark Apples' mlx:
# %pip install mlx
import mlx.core as mx

In [17]:
# Maker an array of floats.
A_numpy = np.random.randn(int(1e6), 10)

A_numpy = A_numpy.copy()
A_Polars = pl.DataFrame(A_numpy)
A_Pandas_numpy = pd.DataFrame(A_numpy)
A_Pandas_arrow = pd.DataFrame(A_numpy, dtype="float32[pyarrow]")
# A_arrow = pa.Table.from_Pandas(A_Pandas_numpy) # no sum method
A_mlx = mx.array(A_numpy)

Candidates currently omited:

1. JAX
2. PyTorch
3. TensorFlow
4. ...?



### Summing Over Columns


In [18]:
%timeit -n 4 -r 2 A_numpy.sum(axis=0)

In [19]:
A_numpy.sum(axis=0).shape

In [20]:
%timeit -n 4 -r 2 A_Polars.sum()

In [21]:
A_Polars.sum().shape

In [22]:
%timeit -n 4 -r 2 A_mlx.sum(axis=0)

In [23]:
A_mlx.sum(axis=0).shape

### 50 Shades of Pandas

Pandas with numpy backend

In [24]:
%timeit -n 4 -r 2 A_Pandas_numpy.sum(axis=0)

In [25]:
A_Pandas_numpy.sum(axis=0).shape

Pandas with arrow backend

In [26]:
%timeit -n 4 -r 2 A_Pandas_arrow.sum(axis=0)

In [27]:
A_Pandas_arrow.sum(axis=0).shape

Pandas with numpy backend, converted to numpy

In [28]:
%timeit -n 4 -r 2 A_Pandas_numpy.values.sum(axis=0)

In [29]:
A_Pandas_numpy.values.sum(axis=0).shape

Pandas with arrow backend, converted to numpy

In [30]:
%timeit -n 4 -r 2 A_Pandas_arrow.values.sum(axis=0)

In [31]:
type(A_Pandas_arrow.values)

In [32]:
A_Pandas_arrow.values.sum(axis=0).shape

Pandas to mlx

In [33]:
%timeit -n 4 -r 2 mx.array(A_Pandas_numpy.values).sum(axis=0)

In [34]:
mx.array(A_Pandas_numpy.values).sum(axis=0).shape

### Summing Over Rows


In [35]:
%timeit -n 4 -r 2 A_numpy.sum(axis=1)

In [36]:
A_numpy.sum(axis=1).shape

In [37]:
%timeit -n 4 -r 2 A_Polars.sum_horizontal()

In [38]:
A_Polars.sum_horizontal().shape

In [39]:
%timeit -n 4 -r 2 A_mlx.sum(axis=1)

In [40]:
A_mlx.sum(axis=1).shape

### 50 Shades of Pandas

Pandas with numpy backend

In [41]:
%timeit -n 4 -r 2 A_Pandas_numpy.sum(axis=1)

Pandas with arrow backend

In [42]:
%timeit -n 4 -r 2 A_Pandas_arrow.sum(axis=1)

Pandas with numpy backend, converted to numpy

In [43]:
%timeit -n 4 -r 2 A_Pandas_numpy.values.sum(axis=1)

Pandas with arrow backend, converted to numpy

In [44]:
%timeit -n 4 -r 2 A_Pandas_arrow.values.sum(axis=1)

Pandas to mlx

In [45]:
%timeit -n 4 -r 2 mx.array(A_Pandas_numpy.values).sum(axis=1)

## Speed Of Import

Polar's `read_X` functions are quite faster than Pandas. 
This is due to better type "guessing" heuristics, and easier mapping between the disk representation and memory representation of the data.

We benchmark by making synthetic data, save it on disk, and reimporting it.

### CSV Format


In [46]:
n_rows = int(1e5)
n_cols = 10
data_Polars = pl.DataFrame(np.random.randn(n_rows,n_cols))
data_Polars.write_csv('data/data.csv', include_header = False)
f"{os.path.getsize('data/data.csv')/1e7:.2f} MB on disk"

Import with Pandas.


In [47]:
%timeit -n2 -r2 data_Pandas = pd.read_csv('data/data.csv', header = None)

Import with Polars.


In [48]:
%timeit -n2 -r2 data_Polars = pl.read_csv('data/data.csv', has_header = False)

What is the ratio of times on your machine?
How many cores do you have?




### Parquet Format


In [49]:
data_Polars.write_parquet('data/data.parquet')
f"{os.path.getsize('data/data.parquet')/1e7:.2f} MB on disk"

In [50]:
%timeit -n2 -r2 data_Pandas = pd.read_parquet('data/data.parquet')

In [51]:
%timeit -n2 -r2 data_Polars = pl.read_parquet('data/data.parquet')

### Feather (Apache IPC) Format


In [52]:
data_Polars.write_ipc('data/data.feather')
f"{os.path.getsize('data/data.feather')/1e7:.2f} MB on disk"

In [53]:
%timeit -n2 -r2 data_Polars = pl.read_ipc('data/data.feather')

In [54]:
%timeit -n2 -r2 data_Pandas = pd.read_feather('data/data.feather')

### Pickle Format

In [55]:
import pickle
pickle.dump(data_Polars, open('data/data.pickle', 'wb'))
f"{os.path.getsize('data/data.pickle')/1e7:.2f} MB on disk"

In [56]:
%timeit -n2 -r2 data_Polars = pickle.load(open('data/data.pickle', 'rb'))

### Summarizing Import

Things to note:

1. The difference in speed is quite large between Pandas vs. Polars.
1. When dealing with CSV's, the function `pl.read_csv` reads in parallel, and has better type guessing heuristics.
2. The difference in speed is quite large between csv vs. parquet and feather, with feather\<parquet\<csv.
1. Feather is the fastest, but larger on disk. Thus good for short-term storage, and parquet for long-term.
1. The fact that pickle isn't the fastest surprised me; but then again, it is not optimized for data.



## Speed Of Join

Because Pandas is built on numpy, people see it as both an in-memory database, and a matrix/array library. With Polars, it is quite clear it is an in-memory database, and not an array processing library (despite having a `pl.dot()` function for inner products). As such, you cannot multiply two Polars dataframes, but you can certainly join then efficiently.

Make some data:


In [57]:
def make_data(n_rows, n_cols):
  data = np.concatenate(
  (
    np.arange(n_rows)[:,np.newaxis], # index
    np.random.randn(n_rows,n_cols), # values
    ),
    axis=1)
    
  return data


n_rows = int(1e6)
n_cols = 10
data_left = make_data(n_rows, n_cols)
data_right = make_data(n_rows, n_cols)

data_left.shape

### Polars Join


In [58]:
data_left_Polars = pl.DataFrame(data_left)
data_right_Polars = pl.DataFrame(data_right)

%timeit -n2 -r2 Polars_joined = data_left_Polars.join(data_right_Polars, on = 'column_0', how = 'left')

### Pandas Join


In [59]:
data_left_Pandas = pd.DataFrame(data_left)
data_right_Pandas = pd.DataFrame(data_right)

%timeit -n2 -r2 Pandas_joined = data_left_Pandas.merge(data_right_Pandas, on = 0, how = 'inner')

## The NYC Taxi Dataset {#sec-nyc_taxi}

Empirical demonstration:
Load the celebrated NYC taxi dataset, filter some rides and get the mean `tip_amount` by `passenger_count`.


In [60]:
path = 'data/NYC' # Data from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
file_names = os.listdir(path)

### Pandas

`df.query()` syntax.

In [61]:
%%time 
taxi_Pandas = pd.read_parquet(path)

query = '''
    passenger_count > 0 and 
    passenger_count < 5 and  
    trip_distance >= 0 and 
    trip_distance <= 10 and 
    fare_amount >= 0 and 
    fare_amount <= 100 and 
    tip_amount >= 0 and 
    tip_amount <= 20 and 
    total_amount >= 0 and 
    total_amount <= 100
    '''.replace('\n', '')
taxi_Pandas.query(query).groupby('passenger_count').agg({'tip_amount':'mean'})

Well, the `df.loc[]` syntax is usually faster than the `query` syntax:


In [62]:
%%time 
taxi_Pandas = pd.read_parquet(path)

ind = (
    taxi_Pandas['passenger_count'].between(1,4) 
    & taxi_Pandas['trip_distance'].between(0,10) 
    & taxi_Pandas['fare_amount'].between(0,100) 
    & taxi_Pandas['tip_amount'].between(0,20) 
    & taxi_Pandas['total_amount'].between(0,100)
)
(
    taxi_Pandas[ind]
    .groupby('passenger_count')
    .agg({'tip_amount':'mean'})
)

### Polars


In [63]:
%%time 

import pyarrow.dataset as ds
dset = ds.dataset("data/NYC", format="parquet")  # define folder as Pyarrow dataset

q = (
    pl.scan_pyarrow_dataset(dset)
    # pl.read_parquet("data/NYC/*.parquet") # will now work because parquet was created with Int32, and not Int64. 
    .filter(
        pl.col('passenger_count').is_between(1,4),
        pl.col('trip_distance').is_between(0,10),
        pl.col('fare_amount').is_between(0,100),
        pl.col('tip_amount').is_between(0,20),
        pl.col('total_amount').is_between(0,100)
    )
    .group_by('passenger_count')
    .agg(pl.col('tip_amount').mean().name.suffix('_mean'))
    )

q.collect()

In [64]:
q.show_graph() # Graphviz has to be installed

Things to note:

1. I only have 2 parquet files. When I run the same with more files, despite my 16GB of RAM, **Pandas will crash my python kernel**.
2. From the query graph I see import is done in parallel, and filtering done at scanning time!
3. Warning: The `pl.scan_paquet()` function will not work with a glob if files are in a remote data lake (e.g. S3). More on that later...







# Preliminaries




## Object Classes

- **Polars Series**: Like a Pandas series. An in-memory array of data, with a name, and a dtype. 

- **Polars Expr**: A Polars series that is not yet computed, and that will be computed when needed.  A Polars Expression can be thought of as:
  1. A Lazy Series: A series that is not yet computed, and that will be computed when needed.
  2. A function: That maps a Polars series to another Polars series (possibly of length 1).

- **Polars LazyFrame**: A collection of Polars Expressions. This is the Polars equivalent of a Spark DataFrame. It is lazy, thus allows query planning.

- **Polars DataFrame**: A collection of Polars Series. This is the Polars equivalent of a Pandas DataFrame. It is eager, and does not allow query planning.
  

::: {.callout-warning}
Not all methods are implemented for all classes. 
In particular, not all `pl.Dataframe` methods are implemented for `pl.LazyFrame` and vice versa. The same goes for `pl.Series` and `pl.Expr`.

This is not because the developers are lazy, but because the API is still being developed, and there are fundamental differences between the classes.

Think about it:

1. Why do we not see a `df.height` attribute for a `pl.LazeFrame`?
2. Why do we not see a `df.sample()` method for a `pl.LazyFrame`?

:::



## Evaluation Engines

Polars seemingly **2 evaluation engines**:

1. **Eager**: This is the default. It is the same as Pandas. When you call a method, the method is immediately executed.
2. **Lazy**: This is the same as Spark. When you call a method, the method is not immediately executed. Instead, a query plan is created. The query plan is then executed when you call `.collect()`.

Why "seemingly" 2? 
Because each engine has it's own subtleties.
For instance, the behavior of the lazy engine may depend on streaming VS non-streaming evaluation, and on the means of loading the data.

1. **Streaming**: This is a special case of lazy evaluation. It is used when you have a large dataset, and you want to process it in chunks. You can then call `.collect(streaming=True)` to process the dataset in chunks. This is useful when you have a dataset that is too large to fit into memory, and you want to process it in chunks. It is also useful when you want to process a dataset in real-time, as it is being generated.
2. **Loading**: Reading multiple parquet files using Polars native readers, may behave slightly different than reading the same files as a Pyarrow dataset (always prefer the native readers, when possible).
  



## Polars dtypes

Polars has its own dtypes, call with `pl.<dtype>`; e.g. `pl.Int32`.
A comprehensive list may be found [here](https://docs.pola.rs/py-Polars/html/reference/datatypes.html).

Here are the most common. 
Note, that unlike Pandas, all are native Polars dtypes, and do not recur to Python objects. 

- Floats: 
  - `pl.Float64`: Arguably, the most frequently used dtype.
- Integers: 
  - `pl.Int64`: The most frequently used integer dtype.
- Booleans: 
  - `pl.Boolean`: As the name suggests. 
- Strings: 
  - `pl.Utf8`: The only string encoding supported by Polars. 
  - `pl.String`: Recently introduced as an alias to `pl.Utf8`.
- Temporal: 
  - `pl.Date`: Date, without time.
  - `pl.Datetime`: Date, with time. 
  - `pl.Time`: Time, without date.
  - `pl.Duration`: Time difference.
- Nulls: 
  - `pl.Null`: Polars equivalent of `None`.
  - `np.nan`: The numpy dtype. Essentially a float, and not as a null.
- Nested:
  - `pl.List`: A list of items.
  - `pl.Array`: Fixed length list. 
  - `pl.Struct`: Think of it as a dict within a frame.



Things to note:

-  Polars has no `Int` dtype, nor `Float`. You must specify the number of bits.
-  Polars recently introduced `pl.String` as an alias for `pl.Utf8`. No other encodings are supported.
-  Polars also supports `np.nan`(!), which is different than its `pl.Null` dtype. `np.nan` is a **float**, and `Null` is a None.
  



## The Polars API

- You will fall in love with it!
- Much more similar to PySpark than to Pandas. The Pandas API is simply not amenable to lazy evaluation. 


### Some Design Principles {#sec-api-principles}

Here are some principles that will help you understand the API:

1. All columns are created equal. There are **no indexing** columns. 

2. Always **remember the class** you are operating on: Is it a series, a dataframe, an expression, or a lazyframe?

5. Operations on the columns of a dataframe will always be part of a **context**. Context may include:
    1. `pl.DataFrame.select()`: This is the most common context. It is used to select columns, and to apply operations on columns.
    2. `pl.DataFrame.with_columns()`: This is used to add columns to a dataframe.
    3. `pl.DataFrame.group_by().agg()`: The `.agg()` context works like a `.select()` context, but it is used to apply operations on sub-groups of rows.
    4. `pl.DataFrame.filter()`: This is used to filter rows using expressions that evaluate to Booleans.

6. Two-word methods are always lower-case, and separated by underscores. E.g: `.is_in()` instead of `.isin()`; `.is_null()` instead of `.isnull()`; `.group_by()` instead of `.groupby()` (starting version 19.0.0). 

7. Polars was designed for operation within **columns**, not within rows. Operations within rows are the exception, and the will have a `_horizontal()` suffix. Examples: `pl.sum_horizontal()`, `pl.mean_horizontal()`, `pl.rolling_sum_horizontal()`.

8. Look for `pl.Expr()` methods so you can chain operations. E.g. `pl.col('a').add(pl.col('b'))` is better than `pl.col('a') + pl.col('b')`; the former can be further chained.
  

### Some Examples of the API

Here is an example to give you a little taste of what the API feels like. 


In [65]:
#| label: Polars-api

# Make some data
polars_frame = pl.DataFrame(make_data(100,4))
polars_frame.limit(5)

::: {.callout-note}
What is the difference between `.head()` and `limit()`?
For eager frames? For lazy frames?
:::



Can you parse the following in your head?

In [66]:
(
  polars_frame
  .rename({'column_0':'group'})
  .with_columns(
    pl.col('group').cast(pl.Int32),
    pl.col('column_1').ge(0).alias('non-negative'),
  )
  .group_by('non-negative')
  .agg(
    pl.col('group').is_between(1,4).sum().alias('one-to-four'),
    pl.col('^column_[0-9]$').mean().name.suffix('_mean'),
  )
)

Ask yourself:

- What is `polars_frame`? Is it an eager or a lazy Polars dataframe?
- Why is `column_1_mean` when `non-negative=false` indeed non negative?
- What is a Polars expression?
- What is a Polars series?
- How did I create the columns `column_1_mean`...`column_4_mean`?
- How would you have written this in Pandas?


In [67]:
#| label: Polars-api-second-example

(
  polars_frame
  .rename({'column_0':'group'})
  .select(
    pl.col('group').mod(2),

    pl.mean_horizontal(
      pl.col('^column_[0-9]$')
    )
    .name.suffix('_mean')
  )
  .filter(
    pl.col('group').eq(1),
    pl.col('column_1_mean').gt(0)
  )
)

Try parsing the following in your head:


In [68]:
polars_frame_2 = (
  pl.DataFrame(make_data(100,1))
  .select(
    pl.col('*').name.suffix('_second')
  )
)



(
  polars_frame
  .join(
    polars_frame_2,
    left_on = 'column_0',
    right_on = 'column_0_second',
    how = 'left',
    validate='1:1'
  )
)


Can you parse the following in your head?


In [69]:
polars_frame_3 = pl.DataFrame(make_data(10,1))

(
  polars_frame.lazy()
  .select(
    pl.col('*').name.suffix('_first')
  )
  .with_context(
    polars_frame_3.lazy()
    .select(
      pl.col('*').name.suffix('_third')
    ) 
  )
  .select(
    'column_0_first',

    pl.when(
      pl.col('column_0_first').mod(2).eq(0)
      )
    .then(
      pl.lit(1)
      )
    .otherwise(
      polars_frame_3
      .select(
        pl.last().min()
        )
      )
  )
  .collect(streaming=True)
)

## Getting Help

Before we dive in, you should be aware of the following references for further help:

1.  A [github page](https://github.com/pola-rs/Polars). It is particular important to subscribe to [release updates](https://github.com/pola-rs/Polars/releases). 
2.  A [user guide](https://pola-rs.github.io/Polars-book/user-guide/index.html).
3.  A very active community on [Discord](https://discord.gg/4UfP5cfBE7).
4.  The [API reference](https://pola-rs.github.io/Polars/py-Polars/html/reference/index.html).
5.  A Stack-Overflow [tag](https://stackoverflow.com/questions/tagged/python-Polars).
6.  Cheat-sheet for [Pandas users](https://www.rhosignal.com/posts/Polars-Pandas-cheatsheet/).

**Warning**: Be careful of AI assistants such as Github-Copilot, TabNine, etc. 
Polars is still very new, and they may give you Pandas completions instead of Polars.





# Polars Series {#sec-series}

A Polars series looks a feels a lot like a Pandas series.
You usually will try to avoid them because: (a) Your objects will usually be in frames, not in series. (b) If in frames, you will be calling `pl.Expr()` methods, and not `pl.Series()` methods. 
Getting used to Polars series, will thus give you bad intuitions when you move to Polars expressions.
Nevertheless, it is the simples object to learn, so we start there. 


Construct a series


In [70]:
#| label: make-a-series
s = pl.Series("a", [1, 2, 3])
s

Make Pandas series for later comparisons.


In [71]:
#| label: make-a-Pandas-series
s_Pandas = pd.Series([1, 2, 3], name = "a")
s_Pandas

Notice even the printing to Jupiter is different.


In [72]:
#| label: check-series-type
type(s)

In [73]:
#| label: check-Pandas-series-type
type(s_Pandas)

In [74]:
#| label: check-series-dtype
s.dtype

In [75]:
#| label: check-Pandas-series-dtype
s_Pandas.dtype

Renaming a series; will be very useful when operating on dataframe columns.


In [76]:
#| label: rename-series
s.alias("b")

Constructing a series of floats, for later use.


In [77]:
#| label: make-a-float-series
f = pl.Series("a", [1., 2., 3.])
f

In [78]:
#| label: check-float-series-dtype
f.dtype

## Export To Other Python Objects

The current section deals with exports to other python objects, **in memory**. 
See @sec-disk-export for exporting to disk.

Export to Polars DataFrame.

In [79]:
#| label: series-to-Polars-dataframe
s.to_frame() 

Export to Python list. 

In [80]:
#| label: series-to-list
s.to_list()

Export to Numpy array. 

In [81]:
#| label: series-to-numpy
s.to_numpy() # useful for preparing data for learning with scikit-learn

Export to Pandas Series. 

In [82]:
#| label: series-to-Pandas-series
s.to_pandas()

Export to Arrow Array.
Useful for preparing data for learning with XGBoost which supports Arrow.
Maybe Scikit-learn will also support Arrow in the future? Although Scikit-learn [may support Polars](https://github.com/scikit-learn/scikit-learn/issues/25896) before it supports Arrow.

In [83]:
#| label: series-to-arrow
s.to_arrow() 

## Memory Representation of Series

Object size in memory. 
Super useful for profiling.
Will only be available for eager objects; by definitions. 


In [84]:
#| label: series-memory-size
s.estimated_size(unit="b") # 8(bytes) * 3(length)

## Filtering and Subsetting {#sec-filtering-subsetting-series}


In [85]:
s[0] # same as s.__getitem__(0)

To filter, you need to use the `.filter()` method.


In [86]:
#| label: series-filter-with-series
s.filter(pl.Series("a", [True, False, True])) # works

Filtering with the `[` operator will not work:


In [87]:
#| label: series-subset-with-boolean
#| eval: false
s[[True, False, True]]

In [88]:
#| label: series-filter-with-list
s.filter([True, False, True])

In [89]:
#| label: series-limit
s.limit(2)

In [90]:
#| label: series-head
s.head(2)

In [91]:
#| label: series-tail
s.tail(2)

In [92]:
#| label: series-sample
s.sample(2, with_replacement=False)

In [93]:
#| label: series-gather-aka-iloc
s.gather([0, 2]) # same as s[0,2] and Pandas .iloc[[0,2]]

In [94]:
#| label: series-slice
s.slice(1, 2) # same as Pandas .iloc[1:2]

In [95]:
#| label: series-gather-every
s.gather_every(2) # same as Pandas .iloc[::2]

## Aggregations


In [96]:
#| label: series-sum
s.sum()

In [97]:
#| label: series-min
s.min()

In [98]:
#| label: series-arg-min
s.arg_min()

In [99]:
#| label: series-max
s.max()

In [100]:
#| label: series-arg-max
s.arg_max()

In [101]:
#| label: series-mean
s.mean()

In [102]:
#| label: series-median
s.median()

In [103]:
#| label: series-quantile
s.quantile(0.2)

In [104]:
#| label: series-entropy
s.entropy()

In [105]:
#| label: Polars-series-describe
s.describe() 

Polars `pl.series.describe()` is almost the same as Pandas `pd.series.describe()`.


In [106]:
#| label: Pandas-series-describe
s_Pandas.describe()

In [107]:
#| label: series-value-counts
s.value_counts()

## Missing

Thanks to Arrow, Polars has built in missing value support for all(!) dtypes. 
This has been a long awaited feature in the Python data science ecosystem with implications on speed, memory, style and more. 
The [Polars User Guide](https://pola-rs.github.io/Polars-book/user-guide/howcani/missing_data.html) has a great overview of the topic from which we collect some take-homes:

- `np.nan` is also supported along `pl.Null`, but is not considered as a missing value by Polars. This has implications on null counts, statistical aggregations, etc.
- `pl.Null`, and `np.nan`s have their own separate functions for imputing, counting, etc.


In [108]:
m = pl.Series("a", [1, 2, None, np.nan])

In [109]:
m.is_null() # checking for None's. Like Pandas .isna()

In [110]:
m.is_nan() # checking for np.nan's

For comparison with Pandas:


In [111]:
m_Pandas = pd.Series([1, 2, None, np.nan])

In [112]:
m_Pandas.isna()

In [113]:
m_Pandas.isnull() # alias for pd.isna()

### Operating on Missing

We now compare the behavior of Polars to Pandas when operating on missing values.
When interpreting the following remember:

1. For Polars, nan is not missing. It is some unknown number. 
2. For Pandas, nan and Nulls are the same. 


In [114]:
# Polars
m1 = pl.Series("a", [1, None, 2, ]) # python native None
m2 = pl.Series("a", [1, np.nan, 2, ]) # numpy's nan
m3 = pl.Series("a", [1, float('nan'), 2, ]) # python's nan

# Pandas
m4 = pd.Series([1, None, 2 ])
m5 = pd.Series([1, np.nan, 2, ])
m6 = pd.Series([1, float('nan'), 2, ])

In [115]:
[
  m1.sum(), 
  m2.sum(), 
  m3.sum(), 
  m4.sum(), 
  m5.sum(), 
  m6.sum(),
  ]

Things to note:

-   `None` will be ignored by both, which is **unsafe**.
-   `np.nan` will be ignored by Pandas (unsafe), but not by Polars (safe).

Filling missing values; `None` and `np.nan` are treated differently:


In [116]:
#| label: series-fill-null-for-null
m1.fill_null(0)

In [117]:
#| label: series-fill-null-for-nan
m2.fill_null(0)

In [118]:
#| label: series-fill-nan-for-nan
m2.fill_nan(0)

In [119]:
#| label: series-drop-null-for-null
m1.drop_nulls()

In [120]:
#| label: series-drop-nan-for-null
m1.drop_nans()

In [121]:
#| label: series-drop-null-for-nan
m2.drop_nulls()

In [122]:
#| label: series-interpolate-null-for-null
m1.interpolate()

In [123]:
#| label: series-interpolate-null-for-nan
m2.interpolate() # np.nan is not considered missing, so why interpolate?

## Shape Transformations


In [124]:
#| label: series-to-dummies
s.to_dummies()

In [125]:
#| label: series-shift
s.shift(1, fill_value=999)

In [126]:
#| label: series-shift-back
s.shift(-1)

In [127]:
#| label: series-reshape
pl.Series("a",[1,2,3,4]).reshape((2,2))

## Arithmetic Operations

The following will fail for `pl.Series` for will work(!) for `pl.Expr`.


In [128]:
#| label: series-add
#| eval: false

s.add(1)

In [129]:
#| label: series-sub
#| eval: false

s.sub(1)

In [130]:
#| label: series-mul
#| eval: false

s.mul(2)

In [131]:
#| label: series-truediv
#| eval: false

s.truediv(2)

In [132]:
#| label: series-floordiv
#| eval: false

s.floordiv(2)

## Mathematical Transformations


In [133]:
#| label: series-abs
s.abs()

In [134]:
#| label: series-sin
s.sin()

In [135]:
#| label: series-exp
s.exp()

In [136]:
#| label: series-hash
s.hash()

In [137]:
#| label: series-log
s.log()

In [138]:
#| label: series-sqrt
s.sqrt()

## Comparisons


In [139]:
s.equals(pl.Series("a", [1, 2, 3]))

In [140]:
#| label: series-eq
s.eq(2)

In [141]:
#| label: series-ge
s.ge(2)

Clip, aka Winsorize. 


In [142]:
#| label: series-clip
f.clip(lower_bound=1.5,upper_bound=2.5)

In [143]:
#| label: series-round
f.round(2)

In [144]:
#| label: series-ceil
f..add()ceil()

In [145]:
#| label: series-floor
f.floor()

In [146]:
#| label: series-which-max
s.peak_max()

## Search


In [147]:
#| label: series-search-in-list
s.is_in([1, 10])

In [148]:
#| label: series-search-in-range
s.is_between(2, 3, closed='both')

## Apply (map_elements)

Applying your own function. 
Also note the informative error message (introduced in Polars Ver 0.18.0) that will try to recommend a more efficient way to do things.


In [149]:
s.map_elements(lambda x: x + 1)

Are lambda functions really so much slower?


In [150]:
s1 = pl.Series(np.random.randn(int(1e6)))

Adding 1 with apply:


In [151]:
%timeit -n2 -r2 s1.map_elements(lambda x: x + 1)

Adding 1 without apply:


In [152]:
%timeit -n2 -r2 s1+1

## Cumulative Operations


In [153]:
#| label: series-cum-max
s.cum_max()

In [154]:
#| label: series-cumsum
s.cum_sum()

In [155]:
#| label: series-cumprod
s.cum_prod()

In [156]:
#| label: series-ewm
s.ewm_mean(com=0.5)

## Differentiation Operations


In [157]:
#| label: series-diff
s.diff()

In [158]:
#| label: series-pct-change
s.pct_change()

## Windowed Operations


In [159]:
#| label: series-rolling-mean

s.rolling_mean(window_size=2)

In [160]:
#| label: series-rolling-sum

s.rolling_sum(window_size=2)

In [161]:
#| label: series-rolling-map
s.rolling_map(
  sum, 
  window_size=2)

Note:

- `sum` is the `pl.sum()` function. You cannot use arbitrary functions within a `rolling_map()` context. 
- Many rolling functions have been prepared. See the [computations section](https://docs.pola.rs/py-Polars/html/reference/series/computation.html) of the Series class in the official API.



## Logical Aggregations


In [162]:
#| label: make-boolean-series
b = pl.Series("a", [True, True, False])
b.dtype

In [163]:
#| label: series-all
b.all()

In [164]:
#| label: series-any
b.any()

In [165]:
#| label: series-not
b.not_()

## Uniques and Duplicates


In [166]:
#| label: series-is-duplicated
s.is_duplicated()

In [167]:
#| label: series-is-unique
s.is_unique()

In [168]:
#| label: series-n-unique
s.unique() # Same as Pandas drop_duplicates()

In [169]:
s.n_unique()

In [170]:
pl.Series([1,2,3,4,1]).unique_counts()

In [171]:
#| label: series-is-first-distinct
s.is_first_distinct() # not sure there is a pl.Expr method 

Notes:

- `is_first_distinct()` has had many name changes in the past. It was `is_first()` in Polars 0.18.0, and `is_first_distinct()` in Polars 0.19.0.

- Do not confuse `.is_first_distinct()` with `.first()`. The former is a logical aggregation, and the latter is a series method. 


In [172]:
#| label: first-counter-examples
(
  pl.DataFrame(pl.Series("a",[1,2,3,1]))
  .select(
    pl.col('a').first()
    )
)

## Casting


In [173]:
s.cast(pl.Int32)

Things to note:

-   `cast()` is Polars' equivalent of Pandas' `astype()`.
-   The dtypes to cast to are **Polars** dtypes. Don't try `s.cast("int32")`, `s.cast(np.int32)`, or `s.cast(pd.int)`
-   For a list of dtypes see the official [documentation](see%20https://pola-rs.github.io/Polars/py-Polars/html/reference/datatypes.html).



Find the most efficient dtype for a series; Like Pandas `pd.to_numeric(..., downcast="..."`).


In [174]:
s.shrink_dtype().dtype # 

Also see [here](http://braaannigan.github.io/software/2022/10/31/Polars-dtype-diet.html).

Shrink the memory allocation to the size of the actual data (in place).


In [175]:
s.shrink_to_fit() 

## Ordering and Sorting


In [176]:
#| label: series-sort
s.sort()

In [177]:
#| label: series-reverse
s.reverse()

In [178]:
#| label: series-rank
s.rank()

In [179]:
#| label: series-arg-sort
s.arg_sort() 

`arg_sort()` returns the indices that would sort the series. Same as R's `order()`.


In [180]:
sorted_s = s[s.arg_sort()]
(s.sort() == sorted_s).all()

In [181]:
#| label: series-shuffle
s.shuffle(seed=1) # random permutation

## Date and Time

There are 4 datetime dtypes in Polars:

1.  **Date**: A date, without hours. Generated with `pl.Date()`.
2.  **Datetime**: Date and hours. Generated with `pl.Datetime()`.
3.  **Time**: Hour of day. Generated with `pl.Time()`.
4.  **Duration**: As the name suggests. Similar to `timedelta` in Pandas. Generated with `pl.Duration()`.

::: {.callout-warning}
Python has a sea of modules that support datetimes. 
A partial list includes: [datetime module](https://docs.python.org/3/library/datetime.html), extensions in [dateutil](https://dateutil.readthedocs.io/en/stable/), [numpy](https://numpy.org/doc/stable/reference/arrays.datetime.html), [Pandas](https://Pandas.pydata.org/Pandas-docs/version/1.1/user_guide/timeseries.html), [arrow](https://arrow.readthedocs.io/en/latest/), the deprecated [scikits.timeseries](https://pytseries.sourceforge.net/) and certainly others. 
Be aware of the dtype you are using, and the accompanying methods.
:::


### Time Range


In [182]:
from datetime import datetime, timedelta

date = (
  pl.datetime_range(
    start = datetime(
      year= 2001, month=2, day=2, hour =2, minute=24, second=12), 
    end = datetime(
      year=2002, month=2, day=5, hour =5, minute=34, second=45),
    interval='1s', 
    eager= True, 
  )
  .sample(10)
)

date

Things to note:

-   How else could I have constructed this series? What other types are accepted as `start` and `end`?
-   `pl.date_range` may return a series of dtype `Date` or `Datetime`. This depens of the granularity of the inputs.


In [183]:
date.dtype

Cast to different time unit. 
May be useful when joining datasets, and the time unit is different.


In [184]:
date.dt.cast_time_unit(time_unit="ms")

Datetime methods are accessed with the `.dt` namespace.


### Extract Time Sub-Units


In [185]:
date.dt.second()

In [186]:
date.dt.minute()

In [187]:
date.dt.hour()

In [188]:
date.dt.day()

In [189]:
date.dt.week()

In [190]:
date.dt.weekday()

In [191]:
date.dt.month()

In [192]:
date.dt.year()

In [193]:
date.dt.ordinal_day() # day in year

In [194]:
date.dt.quarter()

### Durations

Equivalent to Pandas `period` dtype.


In [195]:
diffs = date.diff()
diffs

In [196]:
diffs.dtype

::: {.callout-important}
The extrator of sub-units from a `pl.Duration` has recently changed from `.dt.X()` to `.dt.total_X()`.
:::


In [197]:
diffs.dt.total_seconds()

In [198]:
diffs.dt.total_minutes()

In [199]:
diffs.dt.days()

In [200]:
diffs.dt.total_hours()

### Date Aggregations

Note that aggregating dates, returns a `datetime` type object.


In [201]:
date.dt.max()

In [202]:
date.dt.min()

I never quite undersootd that is the "average day."

In [203]:
date.dt.mean()

In [204]:
date.dt.median()

### Date Transformations


In [205]:
date.dt.offset_by(by="-100y")

Notice the syntax of `offset_by`. It is similar to R's `lubridate` package.


In [206]:
date.dt.offset_by(by="1y2m20d")

In [207]:
date.dt.truncate("1m")

In [208]:
date.dt.round(every="1m")

### From Date to String


In [209]:
date.dt.to_string("%Y-%m-%d")

Or equivalently:

In [210]:
date.dt.strftime("%Y-%m-%d")

### From String to Datetime


In [211]:
sd = pl.Series(
    "date",
    [
        "2021-04-22",
        "2022-01-04 00:00:00",
        "01/31/22",
        "Sun Jul  8 00:34:60 2001",
    ],
)
sd

Parse into `Date` type.


In [212]:
sd.str.to_date(format="%F", strict=False)

Or equivalently:

In [213]:
sd.str.strptime(dtype= pl.Date, format="%F", strict=False)

In [214]:
sd.str.strptime(pl.Date, "%D", strict=False)

Parse into `Datetime` type.


In [215]:
sd.str.to_datetime(format="%F %T", strict=False)

Or equivalently:

In [216]:
sd.str.strptime(pl.Datetime, "%F %T", strict=False)

In [217]:
sd.str.strptime(pl.Datetime, "%a %h %d %T %Y",strict=False)

Parse into `Time` dtype.

In [218]:
sd.str.to_time("%a %h %d %T %Y",strict=False)

In [219]:
sd.str.strptime(pl.Time, "%a %h %d %T %Y", strict=False)

## Strings

String methods are accessed with the `.str` namespace.


In [220]:
#| label: make-string-series
st = pl.Series("a", ["foo", "bar", "baz"])

In [221]:
#| label: string-length
st.str.len_chars() # gets number of chars. In ASCII this is the same as lengths()

In [222]:
#| label: string-concat
st.str.concat("-")

In [223]:
#| label: string-count-matches
st.str.count_matches(pattern= 'o') # count literal matches

In [224]:
#| label: string-contains
st.str.contains("foo|tra|bar") 

In [225]:
#| label: string-contains-regrex
st.str.contains("ba[a-zA-Z]") 

In [226]:
#| label: string-contains-regex-2
st.str.contains("[a-zA-Z]{4,5}") 

In [227]:
#| label: string-count-matches-2
st.str.count_matches(pattern='[a-zA-Z]')

In [228]:
#| label: string-ends-with
st.str.ends_with("oo")

In [229]:
#| label: string-starts-with
st.str.starts_with("fo")

To extract the **first** appearance of a pattern, use `extract`:


In [230]:
#| label: sample-strings
url = pl.Series("a", [
            "http://vote.com/ballon_dor?candidate=messi&ref=Polars",

            "http://vote.com/ballon_dor?candidate=jorginho&ref=Polars",

            "http://vote.com/ballon_dor?candidate=ronaldo&ref=Polars"
            ])

In [231]:
#| label: string-extract
url.str.extract("=([a-zA-Z]+)", 1) 
# "=([a-zA-Z]+)" is read: "match an equality, followed by any number of alphanumerics".

To extract **all** appearances of a pattern, use `extract_all`:


In [232]:
#| label: string-extract-all
url.str.extract_all("=(\w+)") # \w is a shorthand for [a-zA-Z0-9_], i.e., alphanumerics and underscore.

In [233]:
#| label: string-pad-end
st.str.pad_end(8, "*")

In [234]:
#| label: string-pad-start
st.str.pad_start(8, "*")

In [235]:
#| label: string-strip-char-start
st.str.strip_chars_start('f')

In [236]:
#| label: string-strip-char-end
st.str.strip_chars_end('r')

Replacing first appearance of a pattern:


In [237]:
#| label: string-replace
st.str.replace("o+", "ZZ")

Replace all appearances of a pattern:


In [238]:
#| label: string-replace-all
st.str.replace_all("o", "ZZ")

String to list of strings. Number of splits inferred.


In [239]:
#| label: string-split
st.str.split(by="o")

In [240]:
#| label: string-split-inclusive
st.str.split(by="a", inclusive=True)

String to dict of strings. 
Number of **splits** fixed.


In [241]:
#| label: string-split-exact
st.str.split_exact("a", 2)

String to dict of strings. 
Length of **output** fixed.


In [242]:
#| label: string-split-length
st.str.splitn("a", 4)

Strip white spaces.


In [243]:
pl.Series(['   ohh   ','   yeah   ']).str.strip()

In [244]:
#| label: string-to-uppercase
st.str.to_uppercase()

In [245]:
#| label: string-to-lowercase
st.str.to_lowercase()

In [246]:
#| label: string-to-titlecase
st.str.to_titlecase()

In [247]:
#| label: string-zfill
st.str.zfill(5)

In [248]:
#| label: string-slice
st.str.slice(offset=1, length=1)

# Polars (Eager) DataFrames {#sec-dataframes}

Recall :

1.  There is no row index (like R's `data.frame`, `data.table`, and `tibble`; unlike Python's `Pandas`).
2.  Will not accept duplicate column names (unlike Pandas).

## Create

A frame can be created as you would expect. From a dictionary of series, a numpy array, a Pandas sdataframe, or a list of Polars (or Pandas) series, etc.


In [249]:
#| label: make-dataframe


df = pl.DataFrame({
  "integer": [1, 2, 3], 
  "date": [
    (datetime(2022, 1, 1)), 
    (datetime(2022, 1, 2)), 
    (datetime(2022, 1, 3))], 
    "float": [4.0, 5.0, 6.0],
    "string": ["a", "b", "c"]})

## Inspect

Nice HTML printing to iPython. 

In [250]:
df

In [251]:
print(df)

In [252]:
df.glimpse() # useful for wide frames. Similar to R's str() of Pandas's .info()

In [253]:
df.columns

In [254]:
df.shape

In [255]:
df.height # probably more useful than df.shape[0]

In [256]:
df.width

In [257]:
df.schema # similar to Pandas info()

In [258]:
df.with_row_count()

Things to note:

1. `df.schema` and `df.columns` will be available for lazy objects, even before materializing them.
2. `df.height` and `df.shape` will not be available for lazy objects, until they are materialized.

## Intro to Column Operations

::: {.callout-important}
This is probably the most important section of the document.
:::

### Contexts {#sec-contexts}

As discussed in @sec-api-principles, operations on columns will always be done within a **context**.
Pandas fans may think of Polars Context as `.eval()` or `.assign()` Pandas methods.

- `df.with_columns()` to add columns.
- `df.select()` to select columns.
- `df.group_by().agg()` is acutually `.select()` within a `group_by()` context.
- `df.filter()` is a context for filtering rows. 

Select Context:

In [259]:
#| label: select-context
df.select(pl.col("integer").add(1))

In [260]:
#| label: with-colummns-context
df.with_columns(pl.col("integer").add(1))

Group-by context:


In [261]:
#| label: group-by-context
df.group_by("string").agg(pl.col("integer").add(1))

### Column Access

Within a context there are many ways to access a column.
Here are some examples of various ways of adding 1 to the integer column.


In [262]:
#| label: add-1-to-column
#| lst-label: col-ref
#| lst-cap: Referencing a Column in a Select Context
df.select(pl.col("integer").add(1))

In [263]:
df.select(pl.col(["integer"]).add(1))

In [264]:
df.select(pl.col(r"^integer$").add(1))

In [265]:
df.select(pl.col(pl.Int64).add(1))

In [266]:
df.select(pl.first().add(1))

In [267]:
import polars.selectors as cs

df.select(cs.by_name("integer").add(1))

In [268]:
df.select(cs.ends_with("ger").add(1))

In [269]:
df.select(cs.starts_with('int').add(1))

In [270]:
df.select(cs.integer().add(1))

In [271]:
df.select(cs.first().add(1))

In [272]:
df.select(cs.matches(r"^integer$").add(1))

In [273]:
df.select(cs.contains("int").add(1))

In [274]:
df.select(pl.col("integer")+1)

In [275]:
df['integer']+1

In [276]:
df.get_column('integer')+1

The following will not work because a series does not have a `.add(1)` method.

In [277]:
#| eval: false
df['integer'].add(1)
df.get_column('integer').add(1)

::: {.callout-important}
Think: What are the differences between these methods?
:::

My preferred syntax is @col-ref, which is the one I will use in the rest of the document.












## Convert to Other Python Objects

You can always convert your `pl.Series` or `pl.DataFrame` to other Python objects.

To Pandas DataFrame

In [278]:
#| label: dataframe-to-Pandas
df.to_pandas()

To Numpy Array

In [279]:
#| label: dataframe-to-numpy
df.to_numpy()

To List of Polars Series

In [280]:
#| label: dataframe-to-list
df.get_columns() # columns as list of Polars series

To list of tuples

In [281]:
#| label: dataframe-to-list-of-tuples
df.rows() 

To Dict of Polars Series

In [282]:
#| label: dataframe-to-dict-of-series
df.to_dict() # columns as dict of Polars series

To Dict of Python Lists

In [283]:
#| label: dataframe-to-dict-of-lists
df.to_dict(as_series=False) # columns as dict of Polars series

To String Representation (`repr`)

In [284]:
#| label: dataframe-to-repr
df.to_init_repr()

To a Polars Series of Polars Struct

In [285]:
#| label: dataframe-to-struct
df.to_struct()

To PyArrow Table

In [286]:
#| label: dataframe-to-arrow
df.to_arrow()

## Statistical Aggregations


In [287]:
#| label: dataframe-describe
df.describe()

Compare to Pandas: Polars will summarize all columns even if they are not numeric.


In [288]:
#| label: dataframe-describe-pandas
df.to_pandas().describe()

Statistical aggregations operate column-wise (and in parallel!).

In [289]:
#| label: dataframe-max
df.max()

In [290]:
#| label: dataframe-min
df.min()

In [291]:
#| label: dataframe-mean
df.mean()

In [292]:
#| label: dataframe-median
df.median()

In [293]:
#| label: dataframe-quantile
df.quantile(0.1)

In [294]:
#| label: dataframe-sum
df.sum()

Constrast with summation in row:

In [295]:
#| label: dataframe-sum-horizontal
df.with_columns(pl.sum_horizontal('*'))

## Selections {#sec-filtering-subsetting-frames}

1.  If you are used to Pandas, recall there is no index. There is thus no need for `loc` vs. `iloc`, `reset_index()`, etc. See [here](https://pola-rs.github.io/Polars-book/user-guide/howcani/selecting_data/selecting_data_indexing.html) for a comparison of extractors between Polars and Pandas.
2.  Filtering and selection is possible with the `[` operator, or the `filter()` and `select()` methods. The latter is recommended to facilitate query planning (discussed in @sec-query-planning).

### Selecting With Indices

The following are presented for completeness. 
Gnerally, you can, and should, avoid selecting with indices. 
See @sec-selecting-columns for selecting columns, and @sec-filtering-rows for selecting rows.


In [296]:
#| label: single-cell-extraction
df[0,0] # like Pandas .iloc[]

Slicing along rows.

In [297]:
#| label: slice-rows
df[0:1] 

Slicing along columns.


In [298]:
#| label: slice-columns
df[:,0:1]

### Selecting Columns {#sec-selecting-columns}

First- do you want to return a Polars frame or a Polars series?

For a frame:

In [299]:
df.select("integer")

For a series:

In [300]:
df['integer']

How do I know which is which? 

1. You can use `type()`.
2. Notice the dimension of the index in the output. 


Select columns with list of labels


In [301]:
df.select(["integer", "float"])

As of Polars\>=15.0.0, you don't have to pass a list:


In [302]:
df.select("integer", "float")

Column slicing by label

In [303]:
df[:,"integer":"float"]

Note: `df.select()` does not support slicing ranges such as `df.select("integer":"float")`. 

Get a column as a Polars Series.

In [304]:
df.get_column('integer')

Get a column as a Polars series.

In [305]:
df.to_series(0)

In [306]:
df.get_column_index('float')

In [307]:
df.drop("integer")

`df.drop()` not have an `inplace` argument. Use `df.drop_in_place()` instead.



### pl.col()

The `pl.col()` is **super important**. It allows you to select columns in many ways, and provides almost all the methods (i.e. Polars Expressions) you will need to operate on them. 


In [308]:
df.select(pl.col(pl.Int64))

In [309]:
df.select(pl.col(pl.Float64))

In [310]:
df.select(pl.col(pl.Utf8))

Python List of Polars dtypes

In [311]:
df.select(pl.col([pl.Int64, pl.Float64]))

Patterns ("glob")

In [312]:
df.select(pl.col("*")) # same as df.select(pl.all())

Regular Expression. Important! Use `pl.col('^<patterh>$')` to match regular expressions.

In [313]:
df.select(pl.col("^\w{4}$")) 

In [314]:
df.select(pl.col('^.*g$'))

In [315]:
#| label: pl-col-with-regex
df.select(pl.col("^.*te.*$")) # regex matching anything with a "te"

You can use `pl.col()` to exclude columns.

In [316]:
df.select(pl.col("*").exclude("integer"))

In [317]:
df.select(pl.col("*").exclude(pl.Float64))

Exciting! [New API](https://pola-rs.github.io/Polars/py-Polars/html/reference/selectors.html) for column selection. 


In [318]:
#| label: pl-column-selector
import polars.selectors as cs

df.select(cs.starts_with('i'))

In [319]:
#| label: pl-column-selector-set-opeartions

df.select(cs.starts_with('i') | cs.starts_with('d'))

In [320]:
df.select(cs.starts_with('i') | cs.starts_with('d'))

### Selecting Rows By Index


In [321]:
df.limit(2) 

In [322]:
df.head(2)

::: {.callout-important}
`df.limit()` is not the same as `df.head()`. The former does not rely on row indices, and will thus work for lazy objects, while the latter will not.
:::


In [323]:
df.tail(1)

In [324]:
df.take_every(2)

In [325]:
df.slice(offset=1, length=1)

In [326]:
df.sample(1)

Because `.sample()` requires row counts, it will not work for lazy objects.


In [327]:
df.row(1) # get row as tuple

### Selecting Rows By Condition {#sec-filtering-rows}

Aka [Projection](https://en.wikipedia.org/wiki/Projection_(relational_algebra)). 

Enter the `df.filter()` context. 


In [328]:
#| label: filter-context
#| lst-label: filter-context
#| lst-cap: Filtering Rows in a DataFrame
df.filter(pl.col("integer") == 2)

Things to note:

- `df.filter()` is a **Polars Context**. 
- It is a **keep** filter, not a **drop** filter: it will evaluate expressions, and return the rows where the expression does not evaluate to `False`. 
-   The `[` operator does not support indexing with boolean such as `df[df["integer"] == 2]`.
-   The `filter()` method is recommended over `[` by the authors of Polars, to facilitate lazy evaluation (discussed later).


An alternative syntax for equality filtering, known as **constraint** in the Polars documentation.


In [329]:
df.filter(integer = 2) 

AND conditions:

In [330]:
#| label: filter-and
df.filter(
  pl.col('integer').eq(2),
  pl.col('float').gt(10)
)

In [331]:
#| label: filter-and-2
df.filter(
  pl.col('integer').eq(2) &
  pl.col('float').gt(10)
)

OR conditions:

In [332]:
#| label: filter-or
df.filter(
  pl.col('integer').eq(2) |
  pl.col('float').gt(10)
)

::: {.callout-note}
How would you write an AND, or OR condition, without using the comparison methods `.eq()`, `.gt()`, etc.?
:::


### Selecting From Single Item Frame

Say your operation returned a Polars frame with a single float, which you want to manipulate as a Python float:

In [333]:
#| label: single-item-to-python
pl.DataFrame([1]).item() 

## Column Transformations 

1. Transformations are done with Polars Expressions (@sec-expressions) within a **context** (see @sec-contexts).
-   The output column will have the same name as the input, unless you use the `alias()` method to rename it.


In [334]:
#| label: with-columns
df.with_columns(
    pl.col("integer").mul(2),
    pl.col("integer").alias("integer2"),
    integer3 = pl.col("integer").truediv(3)
)

Things to note:

-   You cannot use `[` to assign! This would not have worked `df['integer3'] = df['integer'] * 2`
-   The columns `integer` is multiplied by 2 **in place**, because no `alias` is used.
-   As of Polars version \>15.*.* (I think), you can use `=` to assign. That is how `integer3` is created.
-   The column `integer` is copied, by renaming it to `integer2`.

If a selection returns multiple columns, all will be transformed:

In [335]:
df.with_columns(
    pl.col([pl.Int64,pl.Float64]).mul(2)
)

In [336]:
df.with_columns(
    pl.all().cast(pl.Utf8)
)

You cannot `.alias()` when operating on multiple columns. But you can `.name.suffix()` or `.name.prefix()`.

In [337]:
df.with_columns(
    pl.col([pl.Int64,pl.Float64]).mul(2).name.suffix("_2X")
)

### Conditional Transformation (if-else)


In [338]:
df.with_columns(
    pl.when(
      pl.col("integer").gt(2)
    )
    .then(pl.lit(1))
    .otherwise(pl.col("integer"))
    .alias("new_col")
)

Things to note:

-  The `otherwise()` method is optional. If omitted, the original column will be returned (see next example).
-  `pl.lit(1)` is a Polars expression that returns the literal 1. It may be ommited, but it is good practice to include it for clarity and safety.
-  `pl.col("integer").gt(2)` could have been replaced with `pl.col("integer") > 2`. I like the former because it allows easier composition of conditions.

STOPPED HERE


In [339]:
df.with_columns(
    pl.when(
      pl.col("integer") > 2
    )
    .then(1)
    # .otherwise(pl.col("integer"))
    .alias("new_col")
)

### Python Lambda Functions

Apply your own lambda function.

In [340]:
df.select([pl.col("integer"), pl.col("float")]).apply(lambda x: x[0] + x[1])

As usual, using your own functions may have a very serious toll on performance:


In [341]:
df_big = pl.DataFrame(np.random.randn(1000000, 2), schema=["a", "b"]) # previous versions used columns= instead of schema=

In [342]:
%timeit -n2 -r2 df_big.sum(axis=1)

In [343]:
%timeit -n2 -r2 df_big.apply(lambda x: x[0] + x[1])

How would numpy and Pandas deal with this row-wise summation?


In [344]:
df.shift(1)

In [345]:
df.shift_and_fill(1, 'WOW')

### Numpy Ufuncs

You can use Numpy's universal functions (ufuncs) on Polars frames. 
There is little overhead in using Numpy ufuncs.

- See [here](https://docs.pola.rs/user-guide/expressions/numpy/) to use Numpy   ufuncs in Polars.
- See [here](https://www.w3schools.com/python/numpy/numpy_ufunc_create_function.asp) to create your own Numpy ufunc's.




#### Making Your Own Ufuncs

TODO


## Uniques and Duplicates

Keep uniques; same as `pd.drop_duplicates()`.


In [346]:
#| label: dataframe-unique
df.unique() 

Can be used with column subset

In [347]:
#| label: dataframe-unique-subset
df.unique(["integer", "float"])

In [348]:
df.is_unique()

In [349]:
df.is_duplicated()

In [350]:
df.n_unique()

## Missing


In [351]:
df_with_nulls = df.with_columns(
    pl.Series("missing", [3, None, np.nan]),
)

In [352]:
df_with_nulls.null_count() # same as pd.isnull().sum()

In [353]:
df_with_nulls.drop_nulls() # same as pd.dropna()

Can I also drop `np.nan`'s? 
There is no `drop_nan()` method. 
See [StackOverflow](https://stackoverflow.com/questions/75548444/Polars-dataframe-drop-nans) for workarounds.


In [354]:
df_with_nulls.fill_null(0) # same as pd.fillna(0)

But recall that `None` and `np.nan` are not the same thing.

In [355]:
df_with_nulls.fill_nan(99)

In [356]:
df_with_nulls.interpolate()

## Sorting


In [357]:
df.sort(by=["integer","float"])

In [358]:
df.reverse()

## Groupby

Grouping over categories:

-   `df.partion_by()` will return a list of frames.
-   `df.groupby()` for grouping. Just like Pandas, only parallelized, etc. The output will have the length of the number of groups.
-   `over()` will assign each row the aggregate in the group. Like Pandas `groupby.transform`. The output will have the same length as the input.

Grouping over time:

-   `df.grouby_rolling()` for rolling window grouping, a.k.a. a sliding window. Each row will be assigned the aggregate in the window.
-   `df.groupby_dynamic()` for dynamic grouping. Each period will be assigned the agregate in the period. The output may have more rows than the input.

After grouping:

-   `df.groupby().agg()` for aggregating.
-   `df.groupby().apply()` for applying a function to each group.
-   `df.groupby().count()` for counting.
-   `df.groupby().first()` for getting the first row of each group.
-   ...

See the [API reference](https://pola-rs.github.io/Polars/py-Polars/html/reference/dataframe/groupby.html) for the various options. Also see the [user guide](https://pola-rs.github.io/Polars-book/user-guide/howcani/timeseries/temporal_groupby.html) for more details.


In [359]:
df2 = pl.DataFrame({
    "integer": [1, 1, 2, 2, 3, 3],
    "float": [1.0, 2.0, 3.0, 4.0, 5.0, 6.0],
    "string": ["a", "b", "c", "d", "e", "f"],
    "datetime": [
        (datetime(2022, 1, 4)), 
        (datetime(2022, 1, 4)), 
        (datetime(2022, 1, 4)), 
        (datetime(2022, 1, 9)), 
        (datetime(2022, 1, 9)), 
        (datetime(2022, 1, 9))],
})

In [360]:
df2.partition_by("integer")

In [361]:
groupper = df2.groupby("integer")
groupper.count()

In [362]:
groupper.sum()

Groupby a fixed time window with `df.groupby_dynamic()`:


In [363]:
(
  df2
  .groupby_dynamic(index_column="datetime", every="1d")
  .agg(pl.col("float").sum())
)

If you do not want a single summary per period, rather, a window at each datapoint, use `df.groupby_rolling()`:


In [364]:
(
  df2
  .groupby_rolling(index_column="datetime", period='1d')
  .agg(pl.col("float").sum())
)

### Over

You may be familar with Pandas `groupby().transform()`, which will return a frame with the same row-count as its input. You may be familiar with Postgres SQL [window function](https://www.postgresql.org/docs/current/tutorial-window.html). You may not be familiar with either, and still want to aggregate within group, but propagate the result to all group members. Polars' `over()` is the answer.


In [365]:
df.with_columns(
  pl.col("float").sum().over("string").alias("sum")
).limit(5)

**Careful**: `over()` should follow the aggregation. The following will not fail, but return the wrong result:


In [366]:
df.with_columns(
  pl.col("float").over("string").sum().alias("sum")
).limit(5)

## Joins {#sec-joins}

High level:

-   `df.hstack()` for horizontal concatenation; like Pandas `pd.concat([],axis=1)` or R's `cbind`.
-   `df.vstack()` for vertical concatenation; like Pandas `pd.concat([],axis=0)` or R's `rbind`.
-   `df.merge_sorted()` for vertical stacking, with sorting.
-   `pl.concat()`, which is similar to the previous two, but with memory re-chunking. `pl.concat()` also allows diagonal concatenation, if columns are not shared.
-   `df.extend()` for vertical concatenation, but with memory re-chunking. Similar to `df.vstack().rechunk()`.
-   `df.join()` for joins; like Pandas `pd.merge()` or `df.join()`.

For more on the differences between these methods, see [here](https://www.rhosignal.com/posts/Polars-extend-vstack/).

### hstack


In [367]:
new_column = pl.Series("c", np.repeat(1, df.height))

df.hstack([new_column])

### vstack


In [368]:
df2 = pl.DataFrame({
  "integer": [1, 2, 3], 
  "date": [
    (datetime(2022, 1, 4)), 
    (datetime(2022, 1, 5)), 
    (datetime(2022, 1, 6))], 
    "float":[7.0, 8.0, 9.0],
    "string": ["d", "d", "d"]})


df.vstack(df2)

### Concatenation


In [369]:
pl.concat([df, df2]) 
# equivalent to:
# pl.concat([df, df2], how='vertical', rechunk=True, parallel=True) 

In [370]:
pl.concat([df,new_column.to_frame()], how='horizontal')

### extend


In [371]:
df.extend(df2) # like vstack, but with memory re-chunking. Similar to df.vstack().rechunk()

### merge_sorted


In [372]:
df.merge_sorted(df2, key="integer") # vstacking with sorting.

**Caution**: Joining along rows is possible only if matched columns have the same dtype. Timestamps may be tricky because they may have different time units. Recall that timeunits may be cast before joining using `series.dt.cast_time_unit()`:


In [373]:
#| eval: false
df.with_columns(
    pl.col(pl.Datetime("ns")).dt.cast_time_unit(tu="ms")
)            

If you cannot arrange schema before concatenating, use a diagonal concatenation:


In [374]:
pl.concat(
    [df,new_column.to_frame()], 
    how='diagonal')

### join


In [375]:
df.join(df2, on="integer", how="left")

Things to note:

-   Repeating column names have been suffixed with "\_right".
-   Unlike Pandas, there are no indices. The `on`/`left_on`/`right_on` argument is always required.
-   `how=` may take the following values: 'inner', 'left', 'outer', 'semi', 'anti', 'cross'.
-   The join is super fast, as demonstrated in @sec-motivation above.

### join_asof


In [376]:
df.join_asof(
    df2, 
    left_on="date", 
    right_on='date', 
    by="integer", 
    strategy="backward", 
    tolerance='1w')

Things to note:

-   Yes! `merge_asof()` is also available.
-   The `strategy=` argument may take the following values: 'backward', 'forward'.
-   The `tolerance=` argument may take the following values: '1w', '1d', '1h', '1m', '1s', '1ms', '1us', '1ns'.


## Reshaping

High level:

-   `df.transpose()` as the name suggests.
-   `df.melt()` for wide to long.
-   `df.pivot()` for long to wide.
-   `df.explode()` for breaking strings into rows.
-   `df.unstack()`


In [377]:
df.transpose()

### Wide to Long


In [378]:
# The following example is adapted from Pandas documentation: https://Pandas.pydata.org/docs/reference/api/Pandas.wide_to_long.html

np.random.seed(123)
wide = pl.DataFrame({
    'famid': ["11", "12", "13", "2", "2", "2", "3", "3", "3"],
    'birth': [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'ht1': [2.8, 2.9, 2.2, 2, 1.8, 1.9, 2.2, 2.3, 2.1],
    'ht2': [3.4, 3.8, 2.9, 3.2, 2.8, 2.4, 3.3, 3.4, 2.9]})

wide.head(2)

In [379]:
wide.melt(
  id_vars=['famid', 'birth'], 
  value_vars=['ht1', 'ht2'], 
  variable_name='treatment', 
  value_name='height').sample(5)

Break strings into rows.


In [380]:
wide.explode(columns=['famid']).limit(5)

### Long to Wide


In [381]:
# Example adapted from https://stackoverflow.com/questions/5890584/how-to-reshape-data-from-long-to-wide-format

long = pl.DataFrame({
    'id': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'treatment': ['A', 'A', 'B', 'A', 'A', 'B', 'A', 'A', 'B'],
    'height': [2.8, 2.9, 2.2, 2, 1.8, 1.9, 2.2, 2.3, 2.1]
    })
  
long.limit(5)

In [382]:
long.pivot(
  index='id', # index in the wide format
  columns='treatment', # defines columns in the wide format
  values='height')

In [383]:
long.unstack(step=2) # works like a transpose, and then wrap rows. Change the `step=` to get the feeling. 

## Nested dtypes



## Dataframe in Memory


In [384]:
df.estimated_size(unit="mb")

In [385]:
df.n_chunks() # number of ChunkedArrays in the dataframe

In [386]:
df.rechunk() # ensure contiguous memory layout

In [387]:
df.shrink_to_fit() # reduce memory allocation to actual size

## Processing Multiple Frames Simultanously

Q: What if you want to access a column from frame `df`, when processing frame `df2`?\
A: Just join them.\
Q: What if they are not joinable?\
A: Use a diagonal join. Q: Can't I just add a search-space into the lazy query? A: Ahhh! Use `df.with_context()`.


In [388]:
df3 = pl.Series("blah", [100,2,3]).to_frame()

q = (
    df.lazy()
    .with_context( # add colums of df2 to the search space
        df3.lazy()
        )
    .with_columns(
        pl.col('float').map_dict(remapping={4.0:None}, default=100).fill_null(pl.col('blah').mean()).alias('float2'),
        )
    )

q.collect()

Things to note:

-   `with_context()` is a lazy operation. This is great news, since it means both frames will benefit from query planning, etc.
-   `with_context()` will not copy the data, but rather, add a reference to the data.
-   Why not use `pl.col('blah').mean()` within the `map_dict()`? That is indeed more reasonable. It simply did not work.
-   Try it yourself: Can you use multiple `with_context()`?


### Imputation Example


In [389]:
train_lf = pl.LazyFrame(
    {"feature_0": [-1.0, 0, 1], "feature_1": [-1.0, 0, 1]}
)
test_lf = pl.LazyFrame(
    {"feature_0": [-1.0, None, 1], "feature_1": [-1.0, 0, 1]}
)

(
  test_lf
  .with_context(
    train_lf
    .select(pl.all().suffix("_train")))
    .select(
      pl.col("feature_0")
      .fill_null(
        pl.col("feature_0_train").median()
      )
  )
).collect()

# Polars Expressions {#sec-expressions}

Things to recall about Polars expressions:

1. Think of them as functions, that can be chained, and evaluated (lazyly) within a Polars context such as `df.select()`, `df.with_columns()`, etc.
2. They are not the same as Python functions, and will not work outside of a Polars context.
3. Almost all `pl.Series()` methods are available as Polars expressions. There are however some exceptions.


Because almost all Polars Series methods are available as Polars expressions, we refer the reader to @sec-series for a review of importanta series methods. 
In this section we will focus on exceptions, and some important expressions that are not available for Series.


## Arithmetic






# Polars LazyFrames

# Extensions

## SQL

## Plotting

## ML 
